Downloading and cleaning sea ice extent data

In [5]:
## Import all of the packages I need

from sqlalchemy import create_engine
import numpy as pd
import pandas as pd



In [6]:
## Importing a toolbox so I can extend the number of rows visible in the head() output
from functools import partialmethod

## Merge North Sea Ice extent .csv's into one csv

## head -1 N_01_extent_v3.0.csv > sea_ice_all_combined.csv
##  tail -n +2 -q *.csv >> sea_ice_all_combined.csv

## OR SECOND OPTION

## IN TERMINAL BASH: "cat *sea_ice_all_combined.csv >combined.csv"

In [3]:
## Creat a database in sqlite in terminal 
## .sqlite3 sea_ice_database.db

In [4]:
## Below was pasted into sqlite3 in terminal to create a table in my databse called "meta_data_19_20"

## CREATE TABLE sea_ice_extent_table (
## year FLOAT,
## 	mo TEXT,
## 	datatype TEXT,
##	region TEXT,
##	extent FLOAT,
##	area FLOAT,
##	PRIMARY KEY (year,mo,datatype,region,extent,area)
##	);

In [5]:
## Import my sea_ice data into the table 

## .mode csv
## .import sea_ice_combined.csv sea_ice_extent_table

In [7]:
## Load in my sea ice data from the sqlite database
engine = create_engine("sqlite:///Sea_Ice.db")
sea_ice_df = pd.read_sql('SELECT * FROM sea_ice_extent_table;', engine)

In [8]:
## Load in my AGAGE data from the sqlite database
AGAGE_data_df = pd.read_sql('SELECT * FROM AGAGE_data_table;', engine)

In [8]:
## Another option for viewing the combined csv without going through SQLAlchemy
## Reading the combined csv. Don't need to run this because I did above. 
## sea_ice_df = pd.read_csv("sea_ice_combined.csv")

In [9]:
## Extending the number of rows visible in the head() output
pd.DataFrame.head =partialmethod(pd.DataFrame.head, n=10)

In [10]:
sea_ice_df.head()

,year,mo,datatype,region,extent,area
0,1979.0,1,Goddard,N,15.41,12.41
1,1980.0,1,Goddard,N,14.86,11.94
2,1981.0,1,Goddard,N,14.91,11.91
3,1982.0,1,Goddard,N,15.18,12.19
4,1983.0,1,Goddard,N,14.94,12.01
5,1984.0,1,Goddard,N,14.47,11.68
6,1985.0,1,Goddard,N,14.72,11.69
7,1986.0,1,Goddard,N,14.89,11.88
8,1987.0,1,Goddard,N,14.97,11.90
9,1988.0,1,-9999,N,-9999.00,-9999.00


Before we begin cleaning and merging, we will bring in our AGAGE data so we can get a full view of our data. The third data we will need to bring in will be the El Nino, La Nina, or Neutral catagories.

## Below was pasted into sqlite3 in terminal to create a table in my databse called "meta_data_19_20"

CREATE TABLE AGAGE_data_table (
	time FLOAT,
	MM TEXT,
	YYYY FLOAT,
	CFC11 FLOAT,
	CFC12 FLOAT,
	CFC113 FLOAT,
	CH3CCl3 FLOAT,
	CCl4 FLOAT,
	N20 FLOAT,
	CH4 FLOAT,
	CHCl3 FLOAT,
	PRIMARY KEY (time,MM,YYYY,CFC11,CFC12,CFC113,CH3CCl3,CCl4,N20,CH4,CHCl3)
	);

In [11]:
## The .txt file has some interesting format. Before I make the table I want to read in the csv 
## so I can see what it looks like in a dataframe. I may need to edit it before importing it into the table. 

AGAGE_view_df = pd.read_csv("AGAGE_global_mean_md_copy.txt")

In [12]:
## 
AGAGE_view_df.columns[0]

'     time  MM  YYYY       CFC11      ---        CFC12      ---        CFC113      ---      CH3CCl3      ---         CCl4      ---         N2O      ---          CH4      ---        CHCl3      ---'

In [13]:
## Use a split to seperate the strings by space

clean_columns = AGAGE_view_df.columns[0].split()

In [14]:
clean_columns

['time',
 'MM',
 'YYYY',
 'CFC11',
 '---',
 'CFC12',
 '---',
 'CFC113',
 '---',
 'CH3CCl3',
 '---',
 'CCl4',
 '---',
 'N2O',
 '---',
 'CH4',
 '---',
 'CHCl3',
 '---']

In [15]:
AGAGE_view_df[AGAGE_view_df.columns[0]].values[0].split()

['1978.542',
 '7',
 '1978',
 '147.067',
 '6.349',
 '268.552',
 '11.092',
 '0.000',
 '0.000',
 '71.805',
 '10.458',
 '88.972',
 '1.348',
 '299.316',
 '0.823',
 '0.000',
 '0.000',
 '0.000',
 '0.000']

In [18]:
## type(AGAGE_view_df[AGAGE_view_df.columns[0]].values[0].split()[0])

In [16]:
AGAGE_df_list= []

for i in range(len(AGAGE_view_df)):
    row = AGAGE_view_df[AGAGE_view_df.columns[0]].values[i].split()
    row_number = [float(x) for x in row] ## List comprehension
    AGAGE_df_list.append(row_number)

In [17]:
AGAGE_df_list[0]

[1978.542,
 7.0,
 1978.0,
 147.067,
 6.349,
 268.552,
 11.092,
 0.0,
 0.0,
 71.805,
 10.458,
 88.972,
 1.348,
 299.316,
 0.823,
 0.0,
 0.0,
 0.0,
 0.0]

In [18]:
df = pd.DataFrame(AGAGE_df_list, columns = clean_columns)

In [19]:
df

,time,MM,YYYY,CFC11,---,CFC12,---,CFC113,---,CH3CCl3,---,CCl4,---,N2O,---,CH4,---,CHCl3,---
0,1978.542,7.0,1978.0,147.067,6.349,268.552,11.092,0.0,0.0,71.805,10.458,88.972,1.348,299.316,0.823,0.000,0.000,0.00,0.00
1,1978.625,8.0,1978.0,148.527,5.784,269.862,11.241,0.0,0.0,71.088,10.246,89.489,1.784,299.441,0.561,0.000,0.000,0.00,0.00
2,1978.708,9.0,1978.0,148.925,5.412,271.445,9.476,0.0,0.0,69.845,8.760,89.393,1.479,299.889,0.490,0.000,0.000,0.00,0.00
3,1978.792,10.0,1978.0,149.670,5.189,273.775,9.159,0.0,0.0,70.845,8.261,88.964,1.188,300.557,0.582,0.000,0.000,0.00,0.00
4,1978.875,11.0,1978.0,150.647,4.904,276.780,7.185,0.0,0.0,71.945,7.771,88.941,0.657,300.598,0.467,0.000,0.000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,2017.875,11.0,2017.0,228.388,1.002,511.997,0.308,0.0,0.0,0.000,0.000,78.811,0.467,330.892,0.299,1856.133,32.850,9.95,2.91
473,2017.958,12.0,2017.0,228.371,1.011,511.931,0.423,0.0,0.0,0.000,0.000,78.792,0.454,331.043,0.292,1851.996,34.136,9.98,3.36
474,2018.042,1.0,2018.0,228.197,0.946,511.755,0.519,0.0,0.0,0.000,0.000,78.702,0.408,331.139,0.298,1850.948,35.812,9.88,3.40
475,2018.125,2.0,2018.0,228.116,0.957,511.535,0.580,0.0,0.0,0.000,0.000,78.640,0.399,331.211,0.304,1851.618,36.486,10.23,3.56


In [20]:
df.columns

Index(['time', 'MM', 'YYYY', 'CFC11', '---', 'CFC12', '---', 'CFC113', '---',
       'CH3CCl3', '---', 'CCl4', '---', 'N2O', '---', 'CH4', '---', 'CHCl3',
       '---'],
      dtype='object')

In [21]:
my_new_columns = ['time', 'MM', 'YYYY', 'CFC11', 'CFC11_Unc', 'CFC12', 'CFC12_Unc', 'CFC113', 'CFC113_Unc',
       'CH3CCl3', 'CH3CCl3_Unc', 'CCl4', 'CCl4_Unc', 'N2O', 'N2O_Unc', 'CH4', 'CH4_Unc', 'CHCl3',
       'CHCl3_Unc']

In [22]:
df.columns = my_new_columns

In [23]:
df

,time,MM,YYYY,CFC11,CFC11_Unc,CFC12,CFC12_Unc,CFC113,CFC113_Unc,CH3CCl3,CH3CCl3_Unc,CCl4,CCl4_Unc,N2O,N2O_Unc,CH4,CH4_Unc,CHCl3,CHCl3_Unc
0,1978.542,7.0,1978.0,147.067,6.349,268.552,11.092,0.0,0.0,71.805,10.458,88.972,1.348,299.316,0.823,0.000,0.000,0.00,0.00
1,1978.625,8.0,1978.0,148.527,5.784,269.862,11.241,0.0,0.0,71.088,10.246,89.489,1.784,299.441,0.561,0.000,0.000,0.00,0.00
2,1978.708,9.0,1978.0,148.925,5.412,271.445,9.476,0.0,0.0,69.845,8.760,89.393,1.479,299.889,0.490,0.000,0.000,0.00,0.00
3,1978.792,10.0,1978.0,149.670,5.189,273.775,9.159,0.0,0.0,70.845,8.261,88.964,1.188,300.557,0.582,0.000,0.000,0.00,0.00
4,1978.875,11.0,1978.0,150.647,4.904,276.780,7.185,0.0,0.0,71.945,7.771,88.941,0.657,300.598,0.467,0.000,0.000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,2017.875,11.0,2017.0,228.388,1.002,511.997,0.308,0.0,0.0,0.000,0.000,78.811,0.467,330.892,0.299,1856.133,32.850,9.95,2.91
473,2017.958,12.0,2017.0,228.371,1.011,511.931,0.423,0.0,0.0,0.000,0.000,78.792,0.454,331.043,0.292,1851.996,34.136,9.98,3.36
474,2018.042,1.0,2018.0,228.197,0.946,511.755,0.519,0.0,0.0,0.000,0.000,78.702,0.408,331.139,0.298,1850.948,35.812,9.88,3.40
475,2018.125,2.0,2018.0,228.116,0.957,511.535,0.580,0.0,0.0,0.000,0.000,78.640,0.399,331.211,0.304,1851.618,36.486,10.23,3.56


## Create and import that CSV into a sqlite3 table within the Ice_Data.db 

CREATE TABLE AGAGE_data_table (
	time FLOAT,
	MM TEXT,
	YYYY FLOAT,
	CFC11 FLOAT,
	CFC11_Unc FLOAT,
	CFC12 FLOAT,
	CFC12_Unc FLOAT,
	CFC113 FLOAT,
	CFC113_Unc FLOAT,
	CH3CCl3 FLOAT,
	CH3CCl3_Unc FLOAT,
	CCl4 FLOAT,
	CCl4_Unc FLOAT,
	N20 FLOAT,
	N20_Unc FLOAT,
	CH4 FLOAT,
	CH4_Unc FLOAT,
	CHCl3 FLOAT,
	CHC13_Unc FLOAT,
	PRIMARY KEY (time,MM,YYYY,CFC11,CFC12,CFC113,CH3CCl3,CCl4,N20,CH4,CHCl3)
    );


.mode csv
.import AGAGE_Data.csv AGAGE_data_table

In [24]:
AGAGE_data_df

,time,MM,YYYY,CFC11,CFC11_Unc,CFC12,CFC12_Unc,CFC113,CFC113_Unc,CH3CCl3,CH3CCl3_Unc,CCl4,CCl4_Unc,N20,N20_Unc,CH4,CH4_Unc,CHCl3,CHC13_Unc
0,1979.042,1.0,1979.0,153.245,4.635,283.425,8.146,0.0,0.0,74.405,8.975,91.107,1.915,301.152,0.463,0.000,0.000,0.00,0.00
1,1979.125,2.0,1979.0,152.836,4.366,283.455,7.830,0.0,0.0,74.290,9.603,90.733,2.051,301.215,0.451,0.000,0.000,0.00,0.00
2,1979.208,3.0,1979.0,152.893,4.699,284.052,7.396,0.0,0.0,74.815,9.230,89.528,1.213,300.630,0.256,0.000,0.000,0.00,0.00
3,1979.292,4.0,1979.0,154.167,4.881,285.533,7.573,0.0,0.0,75.941,9.633,90.063,1.294,300.577,0.283,0.000,0.000,0.00,0.00
4,1979.375,5.0,1979.0,155.072,5.354,286.657,8.251,0.0,0.0,76.582,9.587,90.854,1.887,300.672,0.456,0.000,0.000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2017.875,11.0,2017.0,228.388,1.002,511.997,0.308,0.0,0.0,0.000,0.000,78.811,0.467,330.892,0.299,1856.133,32.850,9.95,2.91
467,2017.958,12.0,2017.0,228.371,1.011,511.931,0.423,0.0,0.0,0.000,0.000,78.792,0.454,331.043,0.292,1851.996,34.136,9.98,3.36
468,2018.042,1.0,2018.0,228.197,0.946,511.755,0.519,0.0,0.0,0.000,0.000,78.702,0.408,331.139,0.298,1850.948,35.812,9.88,3.40
469,2018.125,2.0,2018.0,228.116,0.957,511.535,0.580,0.0,0.0,0.000,0.000,78.640,0.399,331.211,0.304,1851.618,36.486,10.23,3.56


In [31]:
## The months are in a different format. Notice that the sea ice months do not contain a decimal and a 0. This will
## need to be changed in order for a successful join. 
## I will use a replace function

## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('1.0' ,'1')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('2.0', '2')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('3.0' ,'3')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('4.0', '4')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('5.0' ,'5')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('6.0', '6')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('7.0' ,'7')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('8.0', '8')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('9.0' ,'9')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('10.0', '10')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('11.0' ,'11')
## AGAGE_data_df['MM'] = AGAGE_data_df['MM'].replace('12.0', '12')

## Ask Rita, Leon or a TA about how to make this replace function mor efficient for large datasets- here I only need to replace
## 12 numbers but in the real world it might be thousands or even millions. 


In [25]:
AGAGE_data_df['MM'] = AGAGE_data_df['MM'].str[:-2]

In [26]:
AGAGE_data_df['YYYY'] = AGAGE_data_df['YYYY'].astype(str).str[:-2]

In [27]:
AGAGE_data_df

,time,MM,YYYY,CFC11,CFC11_Unc,CFC12,CFC12_Unc,CFC113,CFC113_Unc,CH3CCl3,CH3CCl3_Unc,CCl4,CCl4_Unc,N20,N20_Unc,CH4,CH4_Unc,CHCl3,CHC13_Unc
0,1979.042,1,1979,153.245,4.635,283.425,8.146,0.0,0.0,74.405,8.975,91.107,1.915,301.152,0.463,0.000,0.000,0.00,0.00
1,1979.125,2,1979,152.836,4.366,283.455,7.830,0.0,0.0,74.290,9.603,90.733,2.051,301.215,0.451,0.000,0.000,0.00,0.00
2,1979.208,3,1979,152.893,4.699,284.052,7.396,0.0,0.0,74.815,9.230,89.528,1.213,300.630,0.256,0.000,0.000,0.00,0.00
3,1979.292,4,1979,154.167,4.881,285.533,7.573,0.0,0.0,75.941,9.633,90.063,1.294,300.577,0.283,0.000,0.000,0.00,0.00
4,1979.375,5,1979,155.072,5.354,286.657,8.251,0.0,0.0,76.582,9.587,90.854,1.887,300.672,0.456,0.000,0.000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2017.875,11,2017,228.388,1.002,511.997,0.308,0.0,0.0,0.000,0.000,78.811,0.467,330.892,0.299,1856.133,32.850,9.95,2.91
467,2017.958,12,2017,228.371,1.011,511.931,0.423,0.0,0.0,0.000,0.000,78.792,0.454,331.043,0.292,1851.996,34.136,9.98,3.36
468,2018.042,1,2018,228.197,0.946,511.755,0.519,0.0,0.0,0.000,0.000,78.702,0.408,331.139,0.298,1850.948,35.812,9.88,3.40
469,2018.125,2,2018,228.116,0.957,511.535,0.580,0.0,0.0,0.000,0.000,78.640,0.399,331.211,0.304,1851.618,36.486,10.23,3.56


In [28]:
AGAGE_data_df.MM.value_counts()

1     40
3     40
2     40
4     39
8     39
5     39
6     39
10    39
9     39
12    39
7     39
11    39
Name: MM, dtype: int64

In [29]:
## Adjust the year for sea ice
sea_ice_df['year'] = sea_ice_df['year'].astype(str).str[:-2]

In [44]:
sea_ice_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   year       510 non-null    object        
 1   mo         510 non-null    object        
 2   datatype   510 non-null    object        
 3   region     510 non-null    object        
 4   extent     510 non-null    float64       
 5   area       510 non-null    float64       
 6   date_time  510 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 28.0+ KB


In [31]:
## Create datetime object the year and month to int for both tables. 

sea_ice_df["date_time"] = pd.to_datetime(sea_ice_df.year + " " + sea_ice_df.mo, yearfirst= True)
                                    

In [32]:
sea_ice_df

,year,mo,datatype,region,extent,area,date_time
0,1979,1,Goddard,N,15.41,12.41,1979-01-01
1,1980,1,Goddard,N,14.86,11.94,1980-01-01
2,1981,1,Goddard,N,14.91,11.91,1981-01-01
3,1982,1,Goddard,N,15.18,12.19,1982-01-01
4,1983,1,Goddard,N,14.94,12.01,1983-01-01
...,...,...,...,...,...,...,...
505,2016,12,Goddard,N,11.46,9.62,2016-12-01
506,2017,12,Goddard,N,11.74,10.26,2017-12-01
507,2018,12,Goddard,N,11.86,10.45,2018-12-01
508,2019,12,Goddard,N,11.90,10.52,2019-12-01


In [36]:
## sea_ice_df.datetime.dt.year

In [34]:
AGAGE_data_df["date_time"] = pd.to_datetime(AGAGE_data_df.YYYY + " " + AGAGE_data_df.MM, yearfirst= True)
                                   

In [35]:
AGAGE_data_df

,time,MM,YYYY,CFC11,CFC11_Unc,CFC12,CFC12_Unc,CFC113,CFC113_Unc,CH3CCl3,CH3CCl3_Unc,CCl4,CCl4_Unc,N20,N20_Unc,CH4,CH4_Unc,CHCl3,CHC13_Unc,date_time
0,1979.042,1,1979,153.245,4.635,283.425,8.146,0.0,0.0,74.405,8.975,91.107,1.915,301.152,0.463,0.000,0.000,0.00,0.00,1979-01-01
1,1979.125,2,1979,152.836,4.366,283.455,7.830,0.0,0.0,74.290,9.603,90.733,2.051,301.215,0.451,0.000,0.000,0.00,0.00,1979-02-01
2,1979.208,3,1979,152.893,4.699,284.052,7.396,0.0,0.0,74.815,9.230,89.528,1.213,300.630,0.256,0.000,0.000,0.00,0.00,1979-03-01
3,1979.292,4,1979,154.167,4.881,285.533,7.573,0.0,0.0,75.941,9.633,90.063,1.294,300.577,0.283,0.000,0.000,0.00,0.00,1979-04-01
4,1979.375,5,1979,155.072,5.354,286.657,8.251,0.0,0.0,76.582,9.587,90.854,1.887,300.672,0.456,0.000,0.000,0.00,0.00,1979-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2017.875,11,2017,228.388,1.002,511.997,0.308,0.0,0.0,0.000,0.000,78.811,0.467,330.892,0.299,1856.133,32.850,9.95,2.91,2017-11-01
467,2017.958,12,2017,228.371,1.011,511.931,0.423,0.0,0.0,0.000,0.000,78.792,0.454,331.043,0.292,1851.996,34.136,9.98,3.36,2017-12-01
468,2018.042,1,2018,228.197,0.946,511.755,0.519,0.0,0.0,0.000,0.000,78.702,0.408,331.139,0.298,1850.948,35.812,9.88,3.40,2018-01-01
469,2018.125,2,2018,228.116,0.957,511.535,0.580,0.0,0.0,0.000,0.000,78.640,0.399,331.211,0.304,1851.618,36.486,10.23,3.56,2018-02-01


In [37]:
## Merge the two tables, using a left merge, to the year and month. 

sea_ice_AGAGE_merged = pd.merge(left=sea_ice_df, right=AGAGE_data_df, how='inner', left_on='date_time', right_on='date_time')

In [38]:
sea_ice_AGAGE_merged

,year,mo,datatype,region,extent,area,date_time,time,MM,YYYY,...,CH3CCl3,CH3CCl3_Unc,CCl4,CCl4_Unc,N20,N20_Unc,CH4,CH4_Unc,CHCl3,CHC13_Unc
0,1979,1,Goddard,N,15.41,12.41,1979-01-01,1979.042,1,1979,...,74.405,8.975,91.107,1.915,301.152,0.463,0.000,0.000,0.00,0.00
1,1980,1,Goddard,N,14.86,11.94,1980-01-01,1980.042,1,1980,...,81.455,10.163,92.595,2.118,300.952,0.281,0.000,0.000,0.00,0.00
2,1981,1,Goddard,N,14.91,11.91,1981-01-01,1981.042,1,1981,...,87.711,8.382,93.659,1.974,301.406,0.187,0.000,0.000,0.00,0.00
3,1982,1,Goddard,N,15.18,12.19,1982-01-01,1982.042,1,1982,...,92.240,8.623,94.069,1.629,302.441,0.303,0.000,0.000,0.00,0.00
4,1983,1,Goddard,N,14.94,12.01,1983-01-01,1983.042,1,1983,...,96.130,9.041,95.640,2.025,303.693,0.217,0.000,0.000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2013,12,Goddard,N,12.18,10.87,2013-12-01,2013.958,12,2013,...,0.000,0.000,82.693,0.393,327.043,0.291,1820.616,36.421,8.72,2.67
467,2014,12,Goddard,N,12.35,10.89,2014-12-01,2014.958,12,2014,...,0.000,0.000,81.809,0.429,328.103,0.355,1830.214,36.752,8.68,2.53
468,2015,12,Goddard,N,12.04,10.64,2015-12-01,2015.958,12,2015,...,0.000,0.000,80.630,0.407,328.964,0.402,1841.678,37.171,9.10,2.75
469,2016,12,Goddard,N,11.46,9.62,2016-12-01,2016.958,12,2016,...,0.000,0.000,79.476,0.428,329.862,0.388,1849.182,39.295,9.79,3.15


In [43]:
# Export the large dataset to be used for the next iteration of data modeling (MVP)

sea_ice_AGAGE_merged.to_csv('sea_ice_AGAGE_merged.csv', index = False)

In [41]:
## Bring in the El Nino or La Nina table into this notebook to build the dummy columns. 

Nino_Nina_df = pd.read_csv("Nino_Nina_dates.csv")

In [42]:
Nino_Nina_df

,El_Nino,La_Nina,El_Nino_Left,El_Nino_Right,La_Nina_Left,La_Nina_Right
0,1900-1901,NaN,1900,1901,NaN,NaN
1,1902-1903,1903-1904,1902,1903,1903,1904
2,1905-1906,1906-1907,1905,1906,1906,1907
3,NaN,1908-1909,NaN,NaN,1908,1909
4,1911-1912,NaN,1911,1912,NaN,NaN
5,1914-1915,1916-1917,1914,1915,1916,1917
6,1918-1919,1920-1921,1918,1919,1920,1921
7,1923-1924,1924-1925,1923,1924,1924,1925
8,1925-1926,1928-1929,1925,1926,1928,1929
9,1930-1931,1931-1932,1930,1931,1931,1932


In [ ]:
### 

In [ ]:
## Export this new dataframe as a CSV. 
## df.to_csv('AGAGE_Data.csv', index=False)